# Timeseries

In [181]:
'''
    Predict Stock of the day 8 through Timeseries data of previous 1-week by PyTorch
'''

'\n    Predict Stock of the day 8 through Timeseries data of previous 1-week by PyTorch\n'

In [182]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [183]:
torch.manual_seed(0)

In [184]:
# set hyper parameter
seq_length = 7 # 1-week
data_dim = 5   # open price / high price / low price / volume / close price
hidden_dim = 10# output_size of RNN model
output_dim = 1 # after FCL = final prediction
learning_rate = 0.01
iterations = 500

In [185]:
# load dataset
xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1] # reverse order
print(xy.shape)

(732, 5)


In [186]:
# split train and test
train_size = int(len(xy) * 0.8) # 732 * 0.8 => 578

train_set = xy[0: train_size]
test_set = xy[train_size - seq_length:]

In [187]:
# (real value - min value) / (max value - min value) => >=0 and < 1
def minmax_scaler(data):
    numerator = data - np.min(data, 0) # axis=0 => 5 feature
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [188]:
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]   # 1set = 1-week informations (5 feature)
        _y = time_series[i + seq_length, [-1]]  # target = next close price right after 1-week
#         print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

In [189]:
# scaling => price = approx 800 / transaction = approx 100000000 => too much diff of range => 0 ~ 1 scaling
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

In [190]:
# split X and Y by numpy
trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)
print(trainX.shape)

(578, 7, 5)


In [191]:
# numpy to tensor
trainX_tensor, trainY_tensor = torch.FloatTensor(trainX), torch.FloatTensor(trainY)
testX_tensor, testY_tensor = torch.FloatTensor(testX), torch.FloatTensor(testY)

trainX_tensor

tensor([[[2.5307e-01, 2.4507e-01, 2.3398e-01, 4.6608e-04, 2.3204e-01],
         [2.2960e-01, 2.3973e-01, 2.5457e-01, 2.9847e-03, 2.3743e-01],
         [2.4924e-01, 2.4167e-01, 2.4834e-01, 2.5993e-04, 2.2679e-01],
         ...,
         [3.6343e-01, 3.7039e-01, 2.6717e-01, 1.2476e-02, 2.6211e-01],
         [2.5945e-01, 3.1067e-01, 2.7411e-01, 4.5632e-01, 2.7175e-01],
         [2.7601e-01, 2.7831e-01, 1.9847e-01, 5.7017e-01, 1.7810e-01]],

        [[2.2960e-01, 2.3973e-01, 2.5457e-01, 2.9847e-03, 2.3743e-01],
         [2.4924e-01, 2.4167e-01, 2.4834e-01, 2.5993e-04, 2.2679e-01],
         [2.2101e-01, 2.4660e-01, 2.5471e-01, 0.0000e+00, 2.6267e-01],
         ...,
         [2.5945e-01, 3.1067e-01, 2.7411e-01, 4.5632e-01, 2.7175e-01],
         [2.7601e-01, 2.7831e-01, 1.9847e-01, 5.7017e-01, 1.7810e-01],
         [1.5902e-01, 1.7865e-01, 1.4173e-01, 3.9381e-01, 1.6054e-01]],

        [[2.4924e-01, 2.4167e-01, 2.4834e-01, 2.5993e-04, 2.2679e-01],
         [2.2101e-01, 2.4660e-01, 2.5471e-01,

In [192]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        
        # LSTM
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        
        # hidden_dim => output_dim
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)
        
    def forward(self, x):
        # x => (578, 7, 5) / 7 = seq_length / 5 = input_dim
        print('x의 형태 before rnn: ', x.shape)
        x, _status = self.rnn(x)
        tmp = x[:, -1]
        
        # x => (578, 7, 10)
        print('x[0]의 형태: ', x[0].shape)
        print('x[0]의 값: ', x[0])
        print('tmp의 형태: ', tmp.shape)
        print('tmp의 값: ', tmp)
        
        x = self.fc(x[:, -1])
        return x

In [193]:
# 1 => stacking size
net = Net(data_dim, hidden_dim, output_dim, 1)

In [194]:
# loss and optimizer
criterion = torch.nn.MSELoss() # final output => float value
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [195]:
# training
for i in range(iterations):
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print((i+1), '번째 Loss: ', loss.item())
    break

x의 형태 before rnn:  torch.Size([578, 7, 5])
x[0]의 형태:  torch.Size([7, 10])
x[0]의 값:  tensor([[-0.0331,  0.0301,  0.0660, -0.1667, -0.0310, -0.0230,  0.0531,  0.0503,
         -0.0475,  0.0458],
        [-0.0587,  0.0481,  0.1036, -0.2305, -0.0600, -0.0336,  0.0882,  0.0864,
         -0.0750,  0.0727],
        [-0.0727,  0.0594,  0.1231, -0.2565, -0.0766, -0.0363,  0.1088,  0.1095,
         -0.0892,  0.0867],
        [-0.0846,  0.0659,  0.1347, -0.2663, -0.0891, -0.0361,  0.1230,  0.1263,
         -0.0973,  0.0936],
        [-0.0787,  0.0613,  0.1402, -0.2764, -0.0922, -0.0473,  0.1304,  0.1340,
         -0.0939,  0.0943],
        [-0.0924,  0.0595,  0.1554, -0.2584, -0.0887, -0.0622,  0.1293,  0.1397,
         -0.0944,  0.0763],
        [-0.0930,  0.0635,  0.1730, -0.2482, -0.0697, -0.0646,  0.1223,  0.1430,
         -0.0972,  0.0642]], grad_fn=<SelectBackward>)
tmp의 형태:  torch.Size([578, 10])
tmp의 값:  tensor([[-0.0930,  0.0635,  0.1730,  ...,  0.1430, -0.0972,  0.0642],
        [-0.097